# Explorative Notebook
To see how code functions

## Geo-FNO

In [1]:
import torch
import numpy as np

In [6]:
inputX = np.load(r'C:\Users\Noahc\Documents\USYD\PHD\8 - Github\Rev909\data\NACA_Cylinder_X.npy')
inputY = np.load(r'C:\Users\Noahc\Documents\USYD\PHD\8 - Github\Rev909\data\NACA_Cylinder_Y.npy')

In [7]:
print(inputX.shape, inputY.shape)

(2490, 221, 51) (2490, 221, 51)


In [8]:
inputX = torch.tensor(inputX, dtype=torch.float)
inputY = torch.tensor(inputY, dtype=torch.float)
input = torch.stack([inputX, inputY], dim=-1)

In [9]:
input.shape

torch.Size([2490, 221, 51, 2])

In [11]:
inputX[0,:-1,0] - inputX[0,1:,0]

tensor([ 1.5097e+01,  9.1194e+00,  5.5086e+00,  3.3275e+00,  2.0100e+00,
         1.2141e+00,  7.3341e-01,  4.4302e-01,  2.6761e-01,  1.6165e-01,
         9.7644e-02,  5.8982e-02,  3.5628e-02,  3.4930e-02,  3.4245e-02,
         3.3574e-02,  3.2915e-02,  3.2270e-02,  3.1637e-02,  3.1017e-02,
         3.0409e-02,  2.9812e-02,  2.9228e-02,  2.8655e-02,  2.8093e-02,
         2.7542e-02,  2.7002e-02,  2.6472e-02,  2.5953e-02,  2.5445e-02,
         2.4945e-02,  2.4457e-02,  2.3977e-02,  2.3507e-02,  2.3046e-02,
         2.2594e-02,  2.2151e-02,  2.1717e-02,  2.1291e-02,  2.0873e-02,
         2.0464e-02,  2.0063e-02,  1.9669e-02,  1.9284e-02,  1.8906e-02,
         1.8535e-02,  1.8172e-02,  1.7815e-02,  1.7466e-02,  1.7123e-02,
         1.6949e-02,  1.6949e-02,  1.6949e-02,  1.6949e-02,  1.6949e-02,
         1.6949e-02,  1.6949e-02,  1.6949e-02,  1.6949e-02,  1.6949e-02,
         1.6949e-02,  1.6949e-02,  1.6949e-02,  1.6949e-02,  1.6949e-02,
         1.6949e-02,  1.6949e-02,  1.6949e-02,  1.6

In [13]:
import plotly.graph_objects as go

In [22]:
plt = go.Figure()
for i in range(51):
    plt.add_trace(go.Scatter(x=inputX[0,:,i], y=inputY[0,:,i],  mode='markers', marker={'size':2}))
plt.show()

**Geo-FNO datasets are still `stuctured`**. We can interpolate a dataset onto contructed ring of data for the cylinder flow case. But other geometries are a bit difficult.

## Soblev Norms

In [24]:
# load in sample data
dataset = np.load(r"C:\Users\Noahc\Documents\USYD\PHD\0 - Work Space\Markov Studies\2D_NS_Re40.npy")
print(dataset.shape)

(200, 501, 64, 64)


In [44]:
sample_solution = dataset[[0],200,:,:]
x = torch.tensor(sample_solution)
y = torch.tensor(sample_solution)

In [39]:
# soblev 1st
k = 1
balanced = False
size_average = False
a = None
if a == None:
    #a = [1,] * k
    a = [1,1]

In [29]:
(np.linalg.norm(dataset[:,100:,...], axis=(2, 3)) * dataset.shape[-1]).max()

17330.8829836684

In [46]:
nx = x.size()[1]
ny = x.size()[2]
#k = self.k
#balanced = self.balanced
#a = self.a
x = x.view(x.shape[0], nx, ny, -1)
y = y.view(y.shape[0], nx, ny, -1)

k_x = torch.cat((torch.arange(start=0, end=nx//2, step=1),torch.arange(start=-nx//2, end=0, step=1)), 0).reshape(nx,1).repeat(1,ny)
k_y = torch.cat((torch.arange(start=0, end=ny//2, step=1),torch.arange(start=-ny//2, end=0, step=1)), 0).reshape(1,ny).repeat(nx,1)
k_x = torch.abs(k_x).reshape(1,nx,ny,1).to(x.device)
k_y = torch.abs(k_y).reshape(1,nx,ny,1).to(x.device)

x_hat = torch.fft.fftn(x, dim=[1, 2])
y_hat = torch.fft.fftn(y, dim=[1, 2])

if balanced==False:
    weight = 1
    component_k1 = a[0]**2 * (k_x**2 + k_y**2)
    component_k2 = a[1]**2 * (k_x**4 + 2*k_x**2*k_y**2 + k_y**4)
    if k >= 1:
        weight += component_k1
    if k >= 2:
        weight += component_k2
    weight = torch.sqrt(weight)
    #loss = self.rel(x*weight, y*weight)
else:
    raise NotImplementedError
    loss = self.rel(x, y)
    if k >= 1:
        weight = a[0] * torch.sqrt(k_x**2 + k_y**2)
        loss += self.rel(x*weight, y*weight)
    if k >= 2:
        weight = a[1] * torch.sqrt(k_x**4 + 2*k_x**2*k_y**2 + k_y**4)
        loss += self.rel(x*weight, y*weight)
    loss = loss / (k+1)
 
# Note: loss is computed in fourier space

In [55]:
first_soblev = component_k1*x_hat
first_soblev_inverse = torch.fft.ifftn(first_soblev, dim=[1, 2]).real
first_soblev.shape

torch.Size([1, 64, 64, 1])

In [48]:
print('x', x.shape)
print('x_hat', x_hat.shape)
print('first_soblev', first_soblev.shape)
print('first_soblev_inverse', torch.fft.ifftn(first_soblev, dim=[1, 2]).real.shape)

x torch.Size([1, 64, 64, 1])
x_hat torch.Size([1, 64, 64, 1])
first_soblev torch.Size([1, 64, 64, 1])
first_soblev_inverse torch.Size([1, 64, 64, 1])


In [71]:
import plotly.graph_objects as go
import plotly.subplots as subplots
rows=1
cols=4
fig = subplots.make_subplots(rows,cols)
fig.add_trace(go.Heatmap(z=x[0,...,0], showscale=False),1,1)
fig.add_trace(go.Heatmap(z=x_hat[0,...,0].real, showscale=False),1,2)
fig.add_trace(go.Heatmap(z=first_soblev[0,...,0].real, showscale=False),1,3)
fig.add_trace(go.Heatmap(z=first_soblev_inverse[0,...,0], showscale=False),1,4)
for i in range(1, rows*cols + 1):
    fig.update_xaxes(scaleanchor=f"y{i}", scaleratio=1, row=(i-1)//cols + 1, col=(i-1)%cols + 1)
    fig.update_yaxes(constrain='domain', row=(i-1)//cols + 1, col=(i-1)%cols + 1)
fig.show()

According to ChatGPT, these codes refer to the Laplacian and the bi-Laplacian each.

In [72]:
def periodic_derivatives(x, dx=1.0, dy=1.0):
    """
    Compute first and second derivatives with 2nd-order central differences
    and periodic BCs.
    
    x: tensor of shape (B, N, N) or (B, N, N, C)
    dx, dy: grid spacing
    
    Returns:
        dx1, dy1 : first derivatives
        dx2, dy2 : second derivatives
    """

    # shift along x-direction (dim=1)
    x_ip = torch.roll(x, shifts=-1, dims=1)
    x_im = torch.roll(x, shifts=+1, dims=1)

    # shift along y-direction (dim=2)
    y_ip = torch.roll(x, shifts=-1, dims=2)
    y_im = torch.roll(x, shifts=+1, dims=2)

    # First derivatives: (f[i+1] - f[i-1]) / (2*dx)
    dx1 = (x_ip - x_im) / (2.0 * dx)
    dy1 = (y_ip - y_im) / (2.0 * dy)

    # Second derivatives: (f[i+1] - 2f[i] + f[i-1]) / dx^2
    dx2 = (x_ip - 2.0 * x + x_im) / (dx * dx)
    dy2 = (y_ip - 2.0 * x + y_im) / (dy * dy)

    return dx1, dy1, dx2, dy2

In [140]:
dx1, dy1, dx2, dy2 = periodic_derivatives(x, dx=np.pi/32, dy=np.pi/32)

In [78]:
(dx2+dy2).shape

torch.Size([1, 64, 64, 1])

In [93]:
H1_sq = (x**2 + dx1**2 + dy1**2)#.sum(dim=[1,2])

In [89]:
H1_sq.shape

torch.Size([1, 64, 64, 1])

In [142]:
first_soblev_inverse.max()

tensor(162.2606, dtype=torch.float64)

In [141]:
(dx2+dy2).max()

tensor(182.6919, dtype=torch.float64)

In [137]:
(first_soblev_inverse/(dx2+dy2)).mean()

tensor(-0.2524, dtype=torch.float64)

In [143]:
rows=1
cols=4
fig = subplots.make_subplots(rows,cols)
fig.add_trace(go.Heatmap(z=x[0,...,0], showscale=False),1,1)
#fig.add_trace(go.Heatmap(z=H1_sq[0,...,0], showscale=False),1,2)
fig.add_trace(go.Heatmap(z=-(dx2+dy2)[0,...,0], showscale=False),1,2)
fig.add_trace(go.Heatmap(z=(first_soblev_inverse+(dx2+dy2))[0,...,0], showscale=False),1,3)
fig.add_trace(go.Heatmap(z=first_soblev_inverse[0,...,0], showscale=False),1,4)
for i in range(1, rows*cols + 1):
    fig.update_xaxes(scaleanchor=f"y{i}", scaleratio=1, row=(i-1)//cols + 1, col=(i-1)%cols + 1)
    fig.update_yaxes(constrain='domain', row=(i-1)//cols + 1, col=(i-1)%cols + 1)
fig.show()

In [97]:
inverse_weight = torch.fft.ifftn(weight, dim=[1, 2]).real

In [98]:
rows=1
cols=4
fig = subplots.make_subplots(rows,cols)
fig.add_trace(go.Heatmap(z=x[0,...,0], showscale=False),1,1)
fig.add_trace(go.Heatmap(z=H1_sq[0,...,0], showscale=False),1,2)
fig.add_trace(go.Heatmap(z=-(dx2+dy2)[0,...,0], showscale=False),1,3)
fig.add_trace(go.Heatmap(z=inverse_weight[0,...,0], showscale=False),1,4)
for i in range(1, rows*cols + 1):
    fig.update_xaxes(scaleanchor=f"y{i}", scaleratio=1, row=(i-1)//cols + 1, col=(i-1)%cols + 1)
    fig.update_yaxes(constrain='domain', row=(i-1)//cols + 1, col=(i-1)%cols + 1)
fig.show()

In [100]:
x.shape

torch.Size([1, 64, 64, 1])

In [102]:
num_examples=1
diff_norms = torch.norm(x.reshape(num_examples,-1) - y.reshape(num_examples,-1),2 , 1)
y_norms = torch.norm(y.reshape(num_examples,-1), 2, 1)
print(diff_norms.shape)

torch.Size([1])


**Validation Study**

- Train with and without Sobolev Norms, confirm better performance
- Validate that a FDM Sobolev Norm performs similarly
- Only then consider FVM